# SQLAlchemy

We'll start off using the world.sqlite database again.

Rather than import all of sqlalchemy and then need to preface each method with the module name, we're importing a lot of individual methods.

In [1]:
#!/usr/bin/env python
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String
from sqlalchemy import sql, select, join, desc

# Create a Engine object which is our handle into the database.
engine = create_engine('sqlite:///../world.sqlite')

# Connect to the database
conn = engine.connect()

# Read the metadata from the existing database.
#  Since the database already exists and has tables defined, we can create Python objects based on these automatically.
DBInfo=MetaData(engine)

In [2]:
# Auto-create the city object basedon the metadata read into the DBInfo.
city=Table('city', DBInfo, autoload=True)

# Mirroring this command: SELECT Id, Name, Population FROM city LIMIT 10;
query=select([city.c.ID, city.c.Name, city.c.Population]).limit(10)
result = conn.execute(query)
for row in result:
    print(row)

(1, 'Kabul', 1780000)
(2, 'Qandahar', 237500)
(3, 'Herat', 186800)
(4, 'Mazar-e-Sharif', 127800)
(6, 'Rotterdam', 593321)
(7, 'Haag', 440900)
(8, 'Utrecht', 234323)
(9, 'Eindhoven', 201843)
(10, 'Tilburg', 193238)
(11, 'Groningen', 172701)


In [3]:
# Mirroring this command: SELECT Id, Name, Population FROM city WHERE Population > 5000000;

query=select([city.c.ID, city.c.Name, city.c.Population]).where(city.c.Population > 5000000)
result = conn.execute(query)
for row in result:
    print(row)

(206, 'S�o Paulo', 9968485)
(207, 'Rio de Janeiro', 5598953)
(456, 'London', 7285000)
(608, 'Cairo', 6789479)
(939, 'Jakarta', 9604900)
(1024, 'Mumbai (Bombay)', 10500000)
(1025, 'Delhi', 7206704)
(1380, 'Teheran', 6758845)
(1532, 'Tokyo', 7980230)
(1890, 'Shanghai', 9696300)
(1891, 'Peking', 7472000)
(1892, 'Chongqing', 6351600)
(1893, 'Tianjin', 5286800)
(2257, 'Santaf� de Bogot�', 6260862)
(2298, 'Kinshasa', 5064000)
(2331, 'Seoul', 9981619)
(2515, 'Ciudad de M�xico', 8591309)
(2822, 'Karachi', 9269265)
(2823, 'Lahore', 5063499)
(2890, 'Lima', 6464693)
(3320, 'Bangkok', 6320174)
(3357, 'Istanbul', 8787958)
(3580, 'Moscow', 8389200)
(3793, 'New York', 8008278)


In [5]:
# Mirroring this command: 
#   SELECT Id, Name, Population FROM city WHERE Population > 5000000 ORDER BY Population DESC LIMIT 10;

query=select([city.c.ID, city.c.Name, city.c.Population])\
             .where(city.c.Population > 5000000)\
             .order_by(desc(city.c.Population))\
             .limit(10)
result = conn.execute(query)
for row in result:
    print(row)
print(result)

(1024, 'Mumbai (Bombay)', 10500000)
(2331, 'Seoul', 9981619)
(206, 'S�o Paulo', 9968485)
(1890, 'Shanghai', 9696300)
(939, 'Jakarta', 9604900)
(2822, 'Karachi', 9269265)
(3357, 'Istanbul', 8787958)
(2515, 'Ciudad de M�xico', 8591309)
(3580, 'Moscow', 8389200)
(3793, 'New York', 8008278)


In [7]:
# Mirroring this command:
# select Name,Language,Percentage FROM country JOIN countrylanguage ON Code=CountryCode WHERE Language='Portuguese' ORDER BY Percentage DESC;

country=Table('country', DBInfo, autoload=True)
countrylanguage=Table('countrylanguage', DBInfo, autoload=True)

print(select([country.c.Name, countrylanguage.c.Language, countrylanguage.c.Percentage])\
      .select_from(country.join(countrylanguage)).where(countrylanguage.c.Language == 'Portuguese')\
      .order_by(desc(countrylanguage.c.Percentage)))

query=(select([country.c.Name, countrylanguage.c.Language, countrylanguage.c.Percentage])\
      .select_from(country.join(countrylanguage)).where(countrylanguage.c.Language == 'Portuguese')\
      .order_by(desc(countrylanguage.c.Percentage)))
result = conn.execute(query)
for row in result:
    print(row)

SELECT country."Name", countrylanguage."Language", countrylanguage."Percentage" 
FROM country JOIN countrylanguage ON country."Code" = countrylanguage."CountryCode" 
WHERE countrylanguage."Language" = ? ORDER BY countrylanguage."Percentage" DESC
('Portugal', 'Portuguese', Decimal('99.0000000000'))
('Brazil', 'Portuguese', Decimal('97.5000000000'))
('Luxembourg', 'Portuguese', Decimal('13.0000000000'))
('Andorra', 'Portuguese', Decimal('10.8000000000'))
('Guinea-Bissau', 'Portuguese', Decimal('8.1000000000'))
('Paraguay', 'Portuguese', Decimal('3.2000000000'))
('Macao', 'Portuguese', Decimal('2.3000000000'))
('France', 'Portuguese', Decimal('1.2000000000'))
('Canada', 'Portuguese', Decimal('0.7000000000'))
('United States', 'Portuguese', Decimal('0.2000000000'))
('Cape Verde', 'Portuguese', Decimal('0E-10'))
('East Timor', 'Portuguese', Decimal('0E-10'))


## Inserting

I made this following the information on this page: https://docs.sqlalchemy.org/en/13/core/tutorial.html#coretutorial-insert-expressions

In [8]:
print(city.insert())

INSERT INTO city ("ID", "Name", "CountryCode", "District", "Population") VALUES (?, ?, ?, ?, ?)


In [17]:
my_insert=city.insert().values(Name='Gainesville', CountryCode='USA',District='TX',Population=16612)
print(my_insert)

INSERT INTO city ("Name", "CountryCode", "District", "Population") VALUES (?, ?, ?, ?)


In [19]:
result = conn.execute(my_insert)


In [20]:
query=select([city.c.ID, city.c.Name, city.c.District, city.c.Population]).where(city.c.Name.like('Gain%'))
result = conn.execute(query)
for row in result:
    print(row)

(4053, 'Gainesville', 'Florida', 92291)
(5000, 'Gainesville', 'Georgia', 120000)
(1234568, 'Gainesville', 'TX', 16612)
(1234569, 'Gainesville', 'TX', 16612)
(1234570, 'Gainesville', 'TX', 16612)


## Creating database in SQLAlchemy


In [21]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///test.sqlite', echo=True)

In [22]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()

users = Table('users', metadata,
     Column('id', Integer, primary_key=True),
     Column('name', String),
     Column('fullname', String),
     )

addresses = Table('addresses', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', None, ForeignKey('users.id')),
    Column('email_address', String, nullable=False)
    )

In [23]:
metadata.create_all(engine)

2020-06-15 16:34:59,436 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-06-15 16:34:59,437 INFO sqlalchemy.engine.base.Engine ()
2020-06-15 16:34:59,439 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-06-15 16:34:59,439 INFO sqlalchemy.engine.base.Engine ()
2020-06-15 16:34:59,440 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-06-15 16:34:59,441 INFO sqlalchemy.engine.base.Engine ()
2020-06-15 16:34:59,459 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-06-15 16:34:59,460 INFO sqlalchemy.engine.base.Engine ()
2020-06-15 16:34:59,460 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2020-06-15 16:34:59,461 INFO sqlalchemy.engine.base.Engine ()
2020-06-15 16:34:59,462 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2020-06-15 16:34:59,463 INFO sqlalchemy.engine.base.Engine ()
2020-06-15 16:34:5